In [1]:
from xgboost import XGBClassifier
from obspy import read
import os
import sys
import numpy as np
import pandas as pd
from obspy.core import UTCDateTime
from shutil import copyfile
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score


In [2]:
# root = "E:\\obspy_data_processing\\OBSPY\\automatic_triggers_long_20190408_pics_cleared"
# msdroot = "E:\\obspy_data_processing\\OBSPY\\automatic_triggers_long_20190408"
# for filename in os.listdir(root):
#     copyfile(msdroot+"\\"+filename.replace(".png", ""), "copied\\"+filename.replace(".png", ""))

In [2]:
def moving_average(l, N):
    sum = 0
    result = list( 0 for x in l)
 
    for i in range( 0, N ):
        sum = sum + l[i]
        result[i] = sum / (i+1)
 
    for i in range( N, len(l) ):
        sum = sum - l[i-N] + l[i]
        result[i] = sum / N
 
    return np.array(result)
def msd_straight_forward(r):
    shifts = np.arange(len(r))
    msds = np.zeros(shifts.size)    

    for i, shift in enumerate(shifts):
        diffs = r[:-shift if shift else None] - r[shift:]
        sqdist = np.square(diffs).sum(axis=1)
        msds[i] = sqdist.mean()

    return msds

In [3]:
# data_folder = "E:\\obspy_data_processing\\OBSPY\\triggers\\new_triggers_upd"
data_folder = "E:\\obspy_data_processing\\OBSPY\\copied"

In [4]:
dataset = list()
for filename in os.listdir(data_folder)[:48]:
# for filename in os.listdir(data_folder)[:48]:
#     if filename.endswith("-cutted.msd"):
        print(filename)
        stream = read(data_folder+"\\" + filename)
        label = filename[8:23]
        
        date = label[0:4]+label[4:6]+label[6:8]+"T"+label[9:11]+":"+label[11:13]+":"+label[13:15]
#         label = filename[15:29]
#         date = label[0:4]+label[4:6]+label[6:8]+"T"+label[8:10]+":"+label[10:12]+":"+label[12:14]
        try:
            peak_time = UTCDateTime(date)
        except:
            print ("Unexpected error:", sys.exc_info()[0])
            continue
        trace_times = [x.datetime.timestamp() for x in stream[0].times("utcdatetime")]
        trace_times_np = np.asarray(trace_times, dtype=np.int64)
        
        try:
            #ищем ближайший к событию индекс в массиве времён 
            idx = (np.abs(trace_times_np - peak_time.datetime.timestamp())).argmin()
        except:
            print ("Unexpected error:", sys.exc_info()[0])
        for trace in stream:
            l = list(trace.data)
            ma = moving_average(l, 30)
            mean = np.mean(trace.data)
            msd = [abs(mean - x ) for x in trace.data]
            for item in trace.data:
                try:
                    index = np.where(trace.data==item)[0][0]
                except:
                    print ("Unexpected error:", sys.exc_info()[0])

                if index != idx:
                   
                    dataset.append({'X':item, 'SUM':np.sum(trace.data), 'MA': ma[index], 'MSD': msd[index], 'DATE':np.int64(trace_times[index]), 'Y': 0})
                else:
                    dataset.append({'X':item, 'SUM':np.sum(trace.data), 'MA': ma[index], 'MSD': msd[index], 'DATE':np.int64(trace_times[index]), 'Y': 1})
#         break

trigger—20180506T010847770000Z.msd
trigger—20180506T075344000000Z.msd
trigger—20180506T132213620000Z.msd
trigger—20180506T221546510000Z.msd
trigger—20180506T231944510000Z.msd
trigger—20180507T013028040000Z.msd
trigger—20180507T043224730000Z.msd
trigger—20180507T061313560000Z.msd
trigger—20180507T124727350000Z.msd
trigger—20180507T140932100000Z.msd
trigger—20180507T194552220000Z.msd
trigger—20180507T224946600000Z.msd
trigger—20180508T011058800000Z.msd
trigger—20180508T084739940000Z.msd
trigger—20180508T113234810000Z.msd
trigger—20180508T122231200000Z.msd
trigger—20180508T122427840000Z.msd
trigger—20180508T134636050000Z.msd
trigger—20180508T135818600000Z.msd
trigger—20180508T195141080000Z.msd
trigger—20180508T202714200000Z.msd
trigger—20180509T062051600000Z.msd
trigger—20180509T233729980000Z.msd
trigger—20180510T091532610000Z.msd
trigger—20180510T092403430000Z.msd
trigger—20180510T092546600000Z.msd
trigger—20180510T100846160000Z.msd
trigger—20180510T155115790000Z.msd
trigger—20180511T073

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
X_with_date = df[['X', 'SUM', 'MA', 'MSD', 'DATE']]
X = df[['X', 'SUM', 'MA', 'MSD']]
Y = df['Y'].tolist()

In [ ]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state = seed)

In [ ]:
clf = XGBClassifier(scale_pos_weight = 4500)
clf.fit(X_train, y_train, verbose=True)

In [40]:
pred = clf.predict(X_train)
np.sum(pred)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


820173

In [35]:
len(pred)

1653173

In [41]:
np.sum(pred)

820173

In [38]:
np.sum(y_test)

381

In [42]:
accuracy_score(y_train, pred)

0.7557653346998605

In [19]:
len(X)

5009613